<a href="https://colab.research.google.com/github/Smartsie/Playground/blob/master/Char_RNN_Thierry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
This code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself
T Martin, 12/11/2021
"""

'\nThis code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself\nT Martin, 12/11/2021\n'

# Import base libraries

In [2]:
import numpy as np
import pandas as pd
import base64
import requests
import tensorflow.keras as keras
import tensorflow as tf

# Get Text data

In [4]:
def get_text(url):
  '''
    The function get_text will load a txt file at the specified url

    Args:
        url: The url where the text is located. 

    Returns:
        req!: a clean text
  '''

  req = requests.get(url)
  req = req.text
  return req

In [5]:
# Import the big text data
url = "http://f8cho.free.fr/RABELAIS.txt"


In [6]:
text=get_text(url)

In [ ]:
shakespeare_url='https://homl.info/shakespeare'
filepath=tf.keras.utils.get_file('shakespeare.txt',shakespeare_url)
with open(filepath) as f:
  shakespeare_text=f.read()
text=shakespeare_text

# Data preprocessing

In [8]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
encoded=np.array(tokenizer.texts_to_sequences([text]))-0*1

In [9]:
tokenizer.sequences_to_texts(encoded)

['r a b e l a i s \r \n \r \n 1 4 8 3   -   1 5 5 3 \r \n \r \n a r t i s t e   e t   é c r i v a i n \r \n f r a n c a i s ) \r \n n é   e n   1 4 8 3   ( a p p r o x i m a t i v e m e n t ) \r \n d é c é d é   l e   0 7   j a n v i e r   1 5 5 3 \r \n ( à   l \' â g e   d \' e n v i r o n   7 0   a n s ) \r \n \r \n r a b e l a i s   e s t   n é   à   c h i n o n .   0 n   n e   s a i t   p a s   l a   d a t e   p r é c i s e   d e   s a   n a i s s a n c e ,   q u \' o n   à   f i x é e   e n   1 4 8 3 ,   e n   1 4 9 0   e t   e n   1 4 9 5   l a   d a t e   d e   1 4 9 0 ,   d o n n é e   p a r   g u y   p a t i n , \r \n e s t   c e l l e   q u i   n o u s   p a r a î t   d e v o i r   ê t r e   a d o p t é e   d e   p r é f é r e n c e .   1 5 1 9 - 1 5 2 4 ,   l \' e x i s t e n c e   d e   r a b e l a i s   e s t   s a i s i e   p o u r   l a   p r e m i è r e   f o i s   d a n s   u n   d o c u m e n t   c o n t e m p o r a i n \r \n e n   1 5 1 9 . u n   a c t e   d \'   a c

In [ ]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# Transform dataset by using windows
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length, shift=1,drop_remainder=True)
dataset=dataset.flat_map(lambda window: window.batch(window_length))
# Shuffle the dataset with a
batch_size=32
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:,-1],windows[:,1:]))
# One hot encoding
dataset=dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
# prefetching
dataset=dataset.prefetch(1)

In [ ]:
# Define manually the training data
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
text='Thierry'
chars=[i for i in text]
enc.fit([chars])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [ ]:
enc.transform(['T'])

ValueError: ignored

# Model definition

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))    
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',run_eagerly=True)


In [ ]:
history=model.fit(dataset,epochs=20)

Epoch 1/20


ValueError: ignored

In [ ]:
dataset

<PrefetchDataset shapes: ((None, None, 63), (None, None, 33317)), types: (tf.float32, tf.int32)>